**Examples of Collaborative Filtering based Recommendation Systems**...

reference website: https://blog.csdn.net/Jack_yun_feng/article/details/100176399

这个model base的想法是这样的，用类似linear regression来预测每个user对每个product的评分，预测出评分高的product就推荐给user。

网站都在上面，不懂的自己理解

(个人想法是这个逻辑比较很简单，也符合我们project的要求，但是问题是product评分高，并不能说明user会买他，只能说明product好)

一开始先import
然后用pd加载数据，
看看数据长上什么样子

In [6]:
#make necesarry imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
dataset = pd.read_csv("ratings_Beauty.csv")
dataset = dataset[:1000]
dataset.head()

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


然后根据uersID，和porductId分组， 然后计算平均分，再然后下面的初始化，我也不太明白，反正好像全变成0了

In [7]:
# 用户评分数据  groupby 分组  groupby('userId') 根据用户id分组 agg（aggregation聚合）
users_ratings = dataset.groupby('UserId').agg([list])
# print(users_ratings)
# 物品评分数据
items_ratings = dataset.groupby('ProductId').agg([list])
# print(items_ratings)
# 计算全局平均分
global_mean = dataset['Rating'].mean()
# 初始化bu bi
bu = dict(zip(users_ratings.index, np.zeros(len(users_ratings))))
bi = dict(zip(items_ratings.index, np.zeros(len(items_ratings))))


预测function嘛，很简单，平均值加上每个的偏差值就是预测值了

In [8]:
def predict(uid, iid):
    predict_rating = global_mean + bu[uid] + bi[iid]
    return predict_rating

然后把它们都封装在一个class里面，听上去就很厉害，就把要用的function写在一个class，比较好用

-init肯定就存一开始的变量。number_epochs是训练的次数，alpha: learning rate，regression rate，colums就是数据集的columns的名字

-fit function就也是存变量，然后把数据处理一下，分组什么的，users_rating就是每个userId对每个product的rating，好像是这样的，-items_ratings就是给product打过分的userId和rating，最后会叫sgd()来训练整个model。

-然后sgd()就是用梯度下降法(gradient descent)来提高预测的精度，代码自己看，不懂问我也没用。。。

predict跟上面的一样，没上面好讲的


In [16]:
class BaselineCFBySGD(object):

    def __init__(self, number_epochs, alpha, reg, columns=["UserId", "ProductId", "Rating"]):
        # 梯度下降最高迭代次数
        self.number_epochs = number_epochs
        # 学习率
        self.alpha = alpha
        # 正则参数
        self.reg = reg
        # 数据集中user-item-rating字段的名称
        self.columns = columns

    def fit(self, dataset):
        '''
        :param dataset: uid, iid, rating
        :return:
        '''
        self.dataset = dataset
        # 用户评分数据
        self.users_ratings = dataset.groupby(self.columns[0]).agg([list])[[self.columns[1], self.columns[2]]]
        # 物品评分数据
        self.items_ratings = dataset.groupby(self.columns[1]).agg([list])[[self.columns[0], self.columns[2]]]
        # 计算全局平均分
        self.global_mean = self.dataset[self.columns[2]].mean()
        # 调用sgd方法训练模型参数
        self.bu, self.bi = self.sgd()

    def sgd(self):
        '''
        利用随机梯度下降，优化bu，bi的值
        :return: bu, bi
        '''
        # 初始化bu、bi的值，全部设为0
        bu = dict(zip(self.users_ratings.index, np.zeros(len(self.users_ratings))))
        bi = dict(zip(self.items_ratings.index, np.zeros(len(self.items_ratings))))

        for i in range(self.number_epochs):
        #   print("iter%d" % i)
            for uid, iid, real_rating, _ in self.dataset.itertuples(index=False):
                error = real_rating - (self.global_mean + bu[uid] + bi[iid])

                bu[uid] += self.alpha * (error - self.reg * bu[uid])
                bi[iid] += self.alpha * (error - self.reg * bi[iid])

        return bu, bi

    def predict(self, uid, iid):
        predict_rating = self.global_mean + self.bu[uid] + self.bi[iid]
        return predict_rating




加载数据，

先用一千个数据做做实验，数据太多，计算会很慢，

然后用创好object，然后跑跑fit function。就training好了 

然后会弹出可输入值的窗口，复制userId和productId就可以看它预测的值了,自己可以试试，看看预测的怎么样

In [25]:

dataset = pd.read_csv("ratings_Beauty.csv")
dataset = dataset[:1000]
print(dataset.head(5))
bcf = BaselineCFBySGD(20, 0.1, 0.1, ["UserId", "ProductId", "Rating"])
bcf.fit(dataset)
print()

for _ in range(2):
    try:
        # Remove input spaces: .strip()
        uid = input("uid: ").strip()
        iid = input("iid: ").strip()
        print(bcf.predict(uid, iid))
    except:
        print ("Empty input, break")
        break


           UserId   ProductId  Rating   Timestamp
0  A39HTATAQ9V7YF  0205616461     5.0  1369699200
1  A3JM6GV9MNOF9X  0558925278     3.0  1355443200
2  A1Z513UWSAAO0F  0558925278     5.0  1404691200
3  A1WMRR494NWEWV  0733001998     4.0  1382572800
4  A3IAAVS479H7M7  0737104473     1.0  1274227200

Empty input, break


下面部分是为了能够测试整个逻辑的准确性

data_split()就是分割数据，training和testing的数据集百分之八十是trianing，剩下的是testing的，

反正源代码这这样写，看看，随便改

In [18]:
def data_split(dataset, x=0.8, random=False):
    '''
    切分数据集， 这里为了保证用户数量保持不变，将每个用户的评分数据按比例进行拆分
    :param data_path: 数据集路径
    :param x: 训练集的比例，如x=0.8，则0.2是测试集
    :param random: 是否随机切分，默认False
    :return: 用户-物品评分矩阵
    '''
    print("开始切分数据集...")
    # 设置要加载的数据字段的类型
    # dtype = {"userId": np.int32, "movieId": np.int32, "rating": np.float32}
    # 加载数据，我们只用前三列数据，分别是用户ID，电影ID，已经用户对电影的对应评分
    # ratings = pd.read_csv(data_path, dtype=dtype, usecols=range(3))
    ratings = dataset

    testset_index = []
    # 为了保证每个用户在测试集和训练集都有数据，因此按userId聚合
    for uid in ratings.groupby("UserId").any().index:
        user_rating_data = ratings.where(ratings["UserId"]==uid).dropna()
        if random:
            # 因为不可变类型不能被 shuffle方法作用，所以需要强行转换为列表
            index = list(user_rating_data.index)
            np.random.shuffle(index)    # 打乱列表
            _index = round(len(user_rating_data) * x)
            testset_index += list(index[_index:])
        else:
            # 将每个用户的x比例的数据作为训练集，剩余的作为测试集
            index = round(len(user_rating_data) * x)
            testset_index += list(user_rating_data.index.values[index:])

    testset = ratings.loc[testset_index]
    trainset = ratings.drop(testset_index)
    print("完成数据集切分...")
    return trainset, testset


In [ ]:
然后就是用trainset，来训练model

In [21]:
#Training data
trainset, testset = data_split(dataset, random=True)
bcf = BaselineCFBySGD(20, 0.1, 0.1, ["UserId", "ProductId", "Rating"])
bcf.fit(trainset)

开始切分数据集...
完成数据集切分...
iter0
iter1
iter2
iter3
iter4
iter5
iter6
iter7
iter8
iter9
iter10
iter11
iter12
iter13
iter14
iter15
iter16
iter17
iter18
iter19


In [ ]:
然后把预测结果放在这list里面，随意

In [38]:
pred_rating =[]
for uid, iid, real_rating, _ in trainset.itertuples(index=False):
    pred_rating_result = (uid, iid,real_rating,bcf.predict(uid, iid))
    pred_rating.append(pred_rating_result)


再下面，就是计算逻辑准确度来，源代码有两种，一个叫rmse（均方根误差），另一个mae（平均绝对误差），反正都是看误差的。

<img src="img/rmse.png" width="30%">
<img src="img/mae.png" width="30%">

In [20]:
def accuray(predict_results, method="all"):
    '''
    准确性指标计算方法
    :param predict_results: 预测结果，类型为容器，每个元素是一个包含uid,iid,real_rating,pred_rating的序列
    :param method: 指标方法，类型为字符串，rmse或mae，否则返回两者rmse和mae
    :return:
    '''

    def rmse(predict_results):
        '''
        rmse评估指标 (均方根误差)
        :param predict_results:
        :return: rmse
        '''
        length = 0
        _rmse_sum = 0
        for uid, iid, real_rating, pred_rating in predict_results:
            length += 1
            _rmse_sum += (pred_rating - real_rating) ** 2
        return round(np.sqrt(_rmse_sum / length), 4)

    def mae(predict_results):
        '''
        mae评估指标 (平均绝对误差)
        :param predict_results:
        :return: mae
        '''
        length = 0
        _mae_sum = 0
        for uid, iid, real_rating, pred_rating in predict_results:
            length += 1
            _mae_sum += abs(pred_rating - real_rating)
        return round(_mae_sum / length, 4)

    def rmse_mae(predict_results):
        '''
        rmse和mae评估指标
        :param predict_results:
        :return: rmse, mae
        '''
        length = 0
        _rmse_sum = 0
        _mae_sum = 0
        for uid, iid, real_rating, pred_rating in predict_results:
            length += 1
            _rmse_sum += (pred_rating - real_rating) ** 2
            _mae_sum += abs(pred_rating - real_rating)
        return round(np.sqrt(_rmse_sum / length), 4), round(_mae_sum / length, 4)

    if method.lower() == "rmse":
        rmse(predict_results)
    elif method.lower() == "mae":
        mae(predict_results)
    else:
        return rmse_mae(predict_results)

剩下需要做的就是调整 BaselineCFBySGD(20, 0.1, 0.1, ["UserId", "ProductId", "Rating"])里面的参数，来得出准确率的趋势

比如20到100次的训练次数，看误差最小是多少次数，可以用plt来画图，看趋势

或者在误差最小的时候的次数，再调小学习率，更一步降低误差（应该可以。。）


In [40]:
a , b = accuray(pred_rating)
print(a,b)

0.203 0.1513


然后再看看其他什么想弄的， 暂时还没想好。。